# Sistema de recomendación de películas

## Introducción
El presente cuaderno corresponde al código desarrollado para la segunda tarea de la asignatura Inteligencia de Negocios dentro de la Universidad Mayor, correspondiente a un sistema de recomendación de películas basado en el desafío *Netflix Prize*, que concluyó hace poco más de una década.

### 📜 Datos
Los datos con los que se trabajó fueron entregados durante la asignatura y corresponden a aquellos del desafío mencionado. Estos se encontraban en dos archivos:
- **movie_titles.csv**: Archivo con valores separados por comas y filas, en donde cada una de sus 17.770 filas corresponde a una película. Los valores son el identificador de la película, el año de lanzamiento de la película y el nombre de la película.
- **training_set.tar**: Archivo que contiene 17.770 documentos de texto, cada uno de ellos asociado a una de las películas. Cada fila de estos archivos, a excepción de la primera, contiene un identificador de usuario, una calificación entre 1 a 5 y una fecha en la que fue realizada la calificación. La primera fila contiene el identificador de la película, que no es leído dentro del código desarrollado.

###  🎯 Objetivo
Se requiere que, en base a los datos entregados y las calificaciones realizadas por el usuario que consulta, se entregen recomendaciones de películas acorde a sus intereses.

### 🧬 Modelo usado
Se decide usar la librería [LightFM](https://github.com/lyst/lightfm) en Python cuyo principal modelo —basado en la factorización de matrices empleando filtrado colaborativo— es entrenado con una matriz dispersa que contiene las calificaciones de cada usuario para cada una de las películas. Dependiendo de la calidad del modelo entrenado, se obtendrán sugerencias relevantes para cada uno de los usuarios en base a lo que otros usuarios con gustos similares han calificado de manera positiva.

### 🖥️ Entorno bajo el cual se desarrolló
El código utilizado se ejecuta en una computadora con las siguientes características, así que se sugiere usar componentes y software similar como base para lograr replicar los resultados en tiempos razonables:
- AMD Ryzen 5 5600X 6-Core Processor 3.70 GHz
- 32GB RAM
- Almacenamiento SSD
- Windows 10
- Python 3.9

## Desarrollo
### 📚 0. Importación de librerías a usar

In [18]:
import pandas as pd
from dotenv import load_dotenv
import requests
import numpy as np
import os
from tqdm import tqdm
import fastparquet
import scipy.stats as stats
from scipy import sparse
from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
import pickle

plt.rcParams["figure.figsize"] = (12,10)
load_dotenv()

True

### 🎬 1. Preparación de la tabla de películas

#### 1.1 Carga de datos
Para cargar la tabla de películas, simplemente se crea un dataframe de Pandas con el método ``.read_csv``, definiendo que solo se usarán tres columnas para evitar errores o información faltante en caso de que el nombre de alguna película contenga una coma.

In [2]:
movie_titles = pd.read_csv("movie_titles.csv", usecols = range(3),
                            names = ['id', 'year', 'name'],
                            encoding = 'ISO-8859-1')
movie_titles

,id,year,name
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


#### 1.2 Búsqueda de valores faltantes
Con el fin de asegurar que los datos estén en un formato usable para el modelo, además de analizar y corregir errores, se realizan las siguientes operaciones.

In [3]:
for column in movie_titles.columns:
    print("\n\Filas con una celda nula en la columna " + column + ":")
    print(movie_titles[movie_titles[column].isna()].values)


\Filas con una celda nula en la columna id:
[]

\Filas con una celda nula en la columna year:
[[4388 nan 'Ancient Civilizations: Rome and Pompeii']
 [4794 nan 'Ancient Civilizations: Land of the Pharaohs']
 [7241 nan 'Ancient Civilizations: Athens and Greece']
 [10782 nan 'Roti Kapada Aur Makaan']
 [15918 nan 'Hote Hote Pyaar Ho Gaya']
 [16678 nan 'Jimmy Hollywood']
 [17667 nan 'Eros Dance Dhamaka']]

\Filas con una celda nula en la columna name:
[]


Para no tener que eliminar las películas con año nulo de la celda anterior, primero se prueba la obtención del año con la API de The Movie Database, en caso de que esta pudiera entregarnos los años de cada película.

In [4]:
api_key = os.getenv("API_KEY")

for movie in movie_titles[movie_titles['year'].isna()]['name']:
    url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie}"
    response = requests.get(url)
    data = response.json()
    try:
        relase_date = data['results'][0]['release_date']
        if relase_date == '':
            raise
        release_year = relase_date[:4]
        print("🟢 La fecha de lanzamiento de " + movie + " es " + relase_date + 
              ", del año " + release_year + ".")
        movie_titles.loc[movie_titles['name'] == movie, 'year'] = release_year
    except: print("🔴 No se encontró fecha de lanzamiento para " + movie + ".")

🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Rome and Pompeii.
🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Land of the Pharaohs.
🔴 No se encontró fecha de lanzamiento para Ancient Civilizations: Athens and Greece.
🟢 La fecha de lanzamiento de Roti Kapada Aur Makaan es 1974-01-01, del año 1974.
🔴 No se encontró fecha de lanzamiento para Hote Hote Pyaar Ho Gaya.
🟢 La fecha de lanzamiento de Jimmy Hollywood es 1994-03-30, del año 1994.
🔴 No se encontró fecha de lanzamiento para Eros Dance Dhamaka.


Ya que solo se encontró el año de lanzamiento de dos películas, se procede a hacer un ingreso manual del año para las películas restantes en base a los resultados de un motor de búsqueda.

In [5]:
movie_titles.at[4387, 'year'] = 2001
movie_titles.at[4793, 'year'] = 2001
movie_titles.at[7240, 'year'] = 2002
movie_titles.at[15917, 'year'] = 1999
movie_titles.at[17666, 'year'] = 1999

Ahora que están todas las películas con un año en la columna año, se procede a revisar que eso sea así y a definir el tipo de datos a ```np.int32```, cosa que era imposible de hacer antes ya que existían celdas nulas.

In [6]:
movie_titles = movie_titles.astype(dtype = {'id': np.int32, 'year': np.int32})
for column in movie_titles.columns:
    print("\n\Filas con una celda nula en la columna " + column + ":")
    print(movie_titles[movie_titles[column].isna()].values)



\Filas con una celda nula en la columna id:
[]

\Filas con una celda nula en la columna year:
[]

\Filas con una celda nula en la columna name:
[]


### 🔢 2. Carga de la tabla de calificaciones
Para crear la tabla de calificaciones, se probaron diversas combinaciones de funciones y métodos para lograr tener un dataframe con las calificaciones de cada usuario para cada una de las películas, siendo el objetivo que cada fila corresponda a un usuario y cada columna corresponda a una película.

Trabajar con la enorme cantidad de datos obtenidos desde el archivo ``training_set.tar`` generaba estimaciones de uso de memoria imposibles de satisfacer en un principio, así que el código se fue mejorando iterativamente hasta obtener un método de lectura amigable con el hardware utilizado.

La lectura se realizó de la siguiente manera, con un enfoque en hacer el mejor uso de la memoria RAM:

In [10]:
# Carpeta con los archivos .txt, debe ser cambiada en caso de que se desee
ratings_folder = 'c:\\umayor\\training_set'

files = ['\\mv_{}.txt'.format(str(f).zfill(7)) for f in range(1, 17771)]

# Lista donde se almacenarán los dataframes generados en el for a continuación
df_list = []

# Se recorre la carpeta de los archivos .txt y se crea un dataframe por cada uno
for filename in tqdm(files):
    temp = (pd.read_csv(ratings_folder + filename, names = ['user', 'rating', 'year'], skiprows = 1)
                       .astype(dtype = {'user': np.single, 'rating': np.single})
                       .drop(columns = ['year']))
    temp['movie'] = filename.split('.')[0].split('_')[1].lstrip('0')
    ratings_column = temp.groupby(['user', 'movie'])['rating'].sum().unstack()
    df_list.append(ratings_column)
df_list[:2]

100%|██████████| 17770/17770 [01:46<00:00, 166.41it/s]


[movie        1
 user          
 915.0      5.0
 2442.0     3.0
 3321.0     3.0
 4326.0     4.0
 11589.0    3.0
 ...        ...
 2630337.0  5.0
 2630797.0  5.0
 2631796.0  4.0
 2635437.0  4.0
 2647871.0  4.0
 
 [547 rows x 1 columns],
 movie        2
 user          
 11409.0    5.0
 41422.0    4.0
 65932.0    3.0
 69809.0    5.0
 105086.0   5.0
 ...        ...
 2596999.0  4.0
 2606799.0  1.0
 2625420.0  2.0
 2640085.0  5.0
 2648861.0  3.0
 
 [145 rows x 1 columns]]

En la línea 12 del código anterior, se definen los tipos de datos como ``np.single``. Este tipo de datos de 32 bits se almacena en memoria de igual manera que los números de tipo ``float`` del lenguaje de programación C y, bajo las pruebas realizadas, ``np.single`` funciona más rápido que otros tipos de datos de NumPy y Python, llegando a almacenar y leer los datos en memoria en la mitad del tiempo que toma hacerlo con el tipo de datos ``np.int32`` por ejemplo, que es el tipo de datos asignado por defecto.

Cada uno de los dataframes de la lista de dataframes recién poblada corresponde a una columna de la matriz que será creada a continuación. El encabezado de cada columna es el id de una película, para la cual existen varias filas que contienen la calificación de cada uno de los usuarios que entregó una nota, indexadas por el id del usuario.

La siguiente matriz combina cada uno de los dataframes de una columna en un dataframe gigantesco de dimensiones 17.770 x 480.189 (películas x usuarios). Esto requiere de un gran esfuerzo computacional, así que se almacena inmediatamente como un archivo de Apache Parquet en la celda siguiente con el objetivo de no volver a ejecutar la acción nuevamente.

In [12]:
concatenated_df = pd.DataFrame()
concatenated_df = pd.concat(df_list, axis = 1)
concatenated_df = concatenated_df.fillna(0)
concatenated_df

movie,1,2,3,4,5,6,7,8,9,10,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user,,,,,,,,,,,,,,,,,,,,,
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649409.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
concatenated_df.to_parquet('ratings.parquet')

Si los pasos anteriores han sido ejecutados con anterioridad o el archivo parquet fue compartido para evitar el uso innecesario de los recursos, se puede usar la siguiente celda para cargar el dataframe:

In [8]:
concatenated_df = pd.read_parquet('ratings.parquet', engine='fastparquet')
concatenated_df

movie,1,2,3,4,5,6,7,8,9,10,...,17761,17762,17763,17764,17765,17766,17767,17768,17769,17770
user,,,,,,,,,,,,,,,,,,,,,
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649409.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2649421.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Este dataframe contiene millones de celdas con valor ``0.0``, por lo que es una matriz dispersa. Sabiendo esto, se puede comprimir considerablemente la matriz utilizando la función ``csr_matrix()`` del módulo Sparse de la biblioteca SciPy, que es, justamente, el formato de matriz esperado por el modelo de LightFM. En la siguiente celda se ejecuta esa función, además de eliminar la matriz no comprimida de la memoria para liberar espacio útil.

In [14]:
sparse_matrix = sparse.csr_matrix(concatenated_df.to_numpy(dtype = np.single))
del concatenated_df

### ⚖️ 3. Entrenamiento del modelo

Existiendo ya una matriz con los datos comprimidos de las calificaciones, tal como fue explicado en la etapa anterior, esta se divide aleatoriamente en dos partes para poder entrenar el modelo, generando los siguientes conjuntos de datos que fueron separados de forma aleatoria y excluyentes entre ellos:

- 80% de los datos fueron destinados para entrenamiento
- 20% de los datos fueron destinados para pruebas
  
La separación se realiza utilizando la función ``random_train_test_split()`` del módulo ``cross_validation`` de LightFM, que devuelve un arreglo de dos elementos: la matriz con los datos de entrenamiento y la matriz con los datos para la realización de pruebas. Estas dos matrices resultantes se encuentran en el formato de coordenadas de SciPy, ``coo_matrix``, que tienen la característica de poder ser convertidos rápidamente al formato ``csr_matrix``.

In [15]:
train, test = random_train_test_split(sparse_matrix, test_percentage=0.2)

Ahora, el modelo será entrenado y es importante tener presentes los siguientes conceptos:

#### Función de pérdida / loss functions
Para comprender el entrenamiento del modelo, primero definimos que las funciones de pérdida son funciones utilizadas para evaluar el entrenamiento de modelos de machine learning, diciéndole al algoritmo de entrenamiento qué cosas acertó y erró según el tamaño del número que entregue esta función y así vaya ajustando el modelo: mientras más alto el valor resultante, peor entrenado está el modelo (Tseng, 2017).

#### WARP / Weighted Approximate-Rank Pairwise
La función de pérdida *Weighted Approximate-Rank Pairwise*, de siglas WARP, es una implementación estocástica de gradient descent que se puede usar tras el entrenamiento de sistemas de recomendación para mejorar la precisión en k, uno de los indicadores para identificar si el algoritmo entrega buenas recomendaciones o no. En base a los experimentos de Weston, Bengio y Usunier (2011), WARP es más eficiente en el uso de memoria al compararlo con otras funciones de pérdida, ya que permite el entrenamiento de modelos con conjuntos de datos de tamaños mayores que la memoria del dispositivo que se está utilizando.

#### Épocas
Se utilizará la biblioteca LightFM junto al modelo WARP en 5 épocas. Cada época corresponde a un recorrido completo sobre el conjunto de datos, realizado por el algoritmo de entrenamiento. En general, se considera que mientras más épocas ocurran mejor ajustadas estarán las recomendaciones, aunque es posible que la realización de muchas épocas sea un esfuerzo innecesario ya que se puede llegar a un modelo idóneo con pocas épocas.


In [16]:
model = LightFM(loss='warp')
model.fit(train, epochs=5, verbose=True)

Epoch: 100%|██████████| 5/5 [12:55<00:00, 155.03s/it]


Para almacenar el modelo creado, se puede guardar en formato Pickle:

In [19]:
with open('model.pickle', 'wb') as f:
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

Y para reabrir el modelo, se realiza la operación inversa:

In [27]:
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)
model

### 💯 4. Evaluación del modelo

La evaluación del modelo entrenado se realizó de dos maneras, tanto para el conjunto de datos de entrenamiento como para el conjunto de datos de prueba:

#### Precisión en $k$
La precisión en $k$ es uno de los métodos más utilizados para evaluar sistemas de recomendación, y se calcula en base a los aciertos dentro de las primeras $k$ recomendaciones que entregó el modelo tras ordenar las películas sugeridas de forma descendiente. Formalmente se puede definir que, para $r_i$ películas sugeridas por el modelo y $r_j$ películas que el usuario efectivamente valoró de manera positiva, siendo $k$ la cantidad de películas de cada uno de estos conjuntos $r$, la precisión en $k$ de un usuario en particular es:

$$
\text{Precision en }k = \frac{r_i \cap r_j}{k}
$$

Donde $r_i \cap r_j$ es el número total de películas que se encuentran en ambos conjuntos (Kumar, Baskaran, Konjengbam, & Singh, 2021). Esta operación se debe realizar para todas las películas sugeridas para cada usuario, y se calcula el promedio.

#### Puntuación AUC

<p align="center">
<img src="https://upload.wikimedia.org/wikipedia/commons/1/13/Roc_curve.svg" width="300" height="300"><br>
<i>Fuente de la imagen: <a href="https://commons.wikimedia.org/wiki/File:Roc_curve.svg">Wikimedia Commons</a> (<a href="https://creativecommons.org/licenses/by-sa/4.0/deed.en">CC BY-SA 4.0</a>)</i>
</p>


AUC significa *Area Under the (ROC) Curve* y, en el caso de sistemas de recomendación, se usa para medir los aciertos del modelo según la cantidad de valores identificados correctamente en comparación con los falsos positivos, y requiere de la existencia de valores numéricos negativos y positivos para funcionar. En el caso de LightFM, según la documentación, el modelo se encarga de distribuir puntajes bajos como negativos y altos como positivos, así que no es necesario aplicar una función para que las calificaciones (que son todas positivas en este momento) sean separadas en un entero negativo y en un entero positivo.

In [60]:
train_precision = precision_at_k(model, train, k=5).mean()
test_precision = precision_at_k(model, train, test, k=5).mean()
train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.43, test 0.50.
AUC: train 0.96, test 0.96.


Los puntajes obtenidos son bastante altos, en donde la precisión en $k = 5$ dice que, para la mitad de los usuarios, la mitad de las películas que el modelo entrega como las cinco más compatibles en base a las calificaciones hechas por cada uno son películas que efectivamente este usuario vería y calificaría bien. Por otro lado, AUC nos indica que las recomendaciones son pertinentes ya que hay pocos falsos positivos.

### 🧙🏻‍♂️ 5. Predicciones

Para realizar una recomendación de películas a los usuarios, se utiliza el método ``predict()`` del modelo entrenado, en donde se ingresan los parámetros ``user_id`` y el listado de películas a revisar para generar la recomendación. Este método devuelve un arreglo con valores numéricos de tipo flotante, negativos y positivos, en donde aquellos valores positivos de mayor tamaño serán las películas más recomendables para ese usuario en base a la calificación de todos los usuarios.

Creando una función para automatizar la obtención de películas, se realizan estimaciones para un rango de películas, idealmente las 17.770, pero podrían separarse por categoría en el futuro, por ejemplo. Estas estimaciones se ordenan y devuelven, para luego ser impresas en el for definido debajo de la función.

In [28]:
def get_recommendation(model, range_of_movies, user_id, n_items):
    scores = model.predict(user_id, range_of_movies)
    top_items = np.argsort(-scores)[:n_items]
    return top_items

### 🍿 6. Probando el modelo

Debido a que se usa la medición precisión en $k=5$, suena interesante tomar las cinco películas más recomendadas para el usuario 5 y analizar la similitud de ellas con las películas mejor calificadas por ese mismo usuario, con el fin de confirmar el correcto funcionamiento del modelo.

Entre las películas recomendadas existe una película de Ozzy Osbourne, Crown Prince of Darkness (que probablemente sea un concierto o documental). Revisando aquellas películas que el usuario 5 ha calificado en la matriz comprimida, encontramos rápidamente que ya había calificado con un 4 o un 5 otro lanzamiento de Ozzy Osbourne: Live & Loud. Así, queda confirmado que el modelo logra identificar sugerencias pertinentes en base a los gustos de los usuarios y a una alta velocidad.

In [30]:
# Probando la obtención de las 5 películas más recomendables para el usuario 5
for i in get_recommendation(model, [x for x in range(1,17770)], 5, 5):
    print(movie_titles[movie_titles['id'] == i]['name'].values[0])

The In-Laws
Shallow Grave
Ozzy Osbourne: Crown Prince of Darkness
Amityville 1992: It's About Time
Son of Paleface


In [59]:
# Obteniendo las películas calificadas por el usuario 5 con una calificación mayor a 3
for liked_movie in sparse_matrix.getrow(5).indices:
    if sparse_matrix.getcol(liked_movie)[5].data[0] > 3.:
        print(movie_titles[movie_titles['id'] == liked_movie]['name'].values[0]
              + " con nota " + str(sparse_matrix.getcol(liked_movie)[5].data[0]))

Peter Tosh: Stepping Razor: Red X con nota 4.0
Bad Boy Bubby con nota 4.0
Battle Queen 2020 con nota 4.0
Port of Shadows con nota 4.0
The Other Side of Heaven con nota 4.0
To Catch a Thief con nota 4.0
Boz Scaggs: Greatest Hits Live con nota 4.0
Devil Man con nota 4.0
Morrissey: Oye Esteban! con nota 5.0
Ozzy Osbourne: Live & Loud con nota 4.0
Escape from New York con nota 4.0
Alias: Season 2 con nota 4.0
The Job con nota 4.0
Brides of Christ con nota 4.0
Sherlock Holmes Faces Death con nota 5.0
Evita con nota 4.0
Sister My Sister con nota 4.0
The Kung Fu Master con nota 4.0
Dragons: Metal Ages: The Movie con nota 4.0
Baa Baa Black Sheep: Season 1 con nota 4.0
Wilder Napalm con nota 4.0
Law & Order: Criminal Intent: The First Year con nota 4.0
Garfield and Friends: Vol. 3 con nota 4.0
Mariah Carey: #1's con nota 4.0
Witness con nota 4.0
North Shore con nota 4.0
Hot Shot con nota 4.0
Detroit 9000 con nota 4.0
Wild Strawberries con nota 4.0


Es importante considerar que la recomendación de la película de Ozzy Osbourne puede variar debido a que el conjunto de datos de entrenamiento se divide aleatoriamente en el paso 3, donde se utiliza la función ``random_train_test_split()``. Debido a esto, es probable que la situación de la celda superior no siempre sea replicable.

## Referencias

Kumar, N., Baskaran, E., Konjengbam, A., & Singh, M. (2021). Hashtag recommendation for short social media texts using word-embeddings and external knowledge. *Knowledge and Information Systems*, *63*(1), 175-198. doi:10.1007/s10115-020-01515-7

Tseng, G. (6 de diciembre de 2017). *Intro to WARP Loss, automatic differentiation and PyTorch*. Obtenido de Medium: https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a

Weston, J., Bengio, S., & Usunier, N. (2011). Wsabie: Scaling Up To Large Vocabulary Image Annotation. *Proceedings of the Twenty-Second international joint conference on Artificial Intelligence*. *3*, págs. 2764-2770. Barcelona, España: AAAI Press. doi:10.5591/978-1-57735-516-8/IJCAI11-460
